In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/ml-project2/scripts')

In [3]:
!pip install wordninja


In [4]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-631x_two
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-631x_two
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.1.0.dev0-cp36-none-any.whl size=1448361 sha256=1314e5fcc8db25acc603e54f67639302d01d410aff54da72d2ca7dca2bbf033c
  Stored in directory: /tmp/pip-ephem-wheel-cache-13y0uwob/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
Successfully built transformers


In [5]:
from preprocess import preprocess
from bert import *
from data_processing import *
from helper import *

import numpy as np
import torch
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Set all parameters :
SEED = 42
NUM_TRAIN_TWEETS = 100
BATCH_LEN = 20
TRAIN_EPOCHS = 1
TOKEN_LEN = 60  # the maximum is 512
RATIO_TRAIN = 0.8
LEARNING_RATE = 2e-5
EPSILON = 1e-8

In [7]:
# Set all the seeds
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)

In [8]:
# Select the best device available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [9]:
path_to_tweets = '/content/drive/MyDrive/Colab Notebooks/ml-project2/data/twitter-datasets/'#'../data/twitter-datasets/'
pos_path = path_to_tweets + 'train_pos_full.txt'
neg_path = path_to_tweets + 'train_neg_full.txt'
test_path = path_to_tweets + 'test_data.txt'

## Prepare the training data

In [21]:
# Load the training data
df_train = load_train(pos_path, neg_path, NUM_TRAIN_TWEETS)

In [11]:
# Preprocess the training data
#out_pre_train = path_to_tweets + 'pre_train.csv'
#df_train = preprocess(df_train, out_pre_train)

In [12]:
# Tokenize the training data
out_token_train = path_to_tweets + 'token_train.csv'
df_train = bert_tokenize_train(df_train, out_token_train, max_len=TOKEN_LEN)

## Create and train the model

In [13]:
# create the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased').to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [14]:
# split the data into a training set and a validation set
train, validation = train_test_split(df_train, train_size=RATIO_TRAIN, random_state=SEED)

In [15]:
# create the optimizer and the scheduler
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, eps=EPSILON)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train)*TRAIN_EPOCHS)

In [16]:
# train the model
fit_model(model, train, validation, BATCH_LEN, TRAIN_EPOCHS, optimizer, scheduler, device)

Transfer progress:   0%|          | 0/8 [00:00<?, ?it/s]

epoch 0



Transfer progress: 100%|██████████| 8/8 [00:02<00:00,  3.68it/s]

 50%|█████     | 1/2 [00:00<00:00,  6.27it/s]

loss tensor(5.5361, device='cuda:0', grad_fn=<AddBackward0>)



100%|██████████| 1/1 [00:02<00:00,  2.51s/it]

accuracy tensor(0.6250, device='cuda:0')


## Evalutae the test data and create submission

In [17]:
# Load the test data
df_test = load_test(test_path)

In [18]:
# Preprocess the test data
# out_pre_test = path_to_tweets + 'pre_test.csv'
# df_test = preprocess(df_test, out_pre_test)

In [19]:
# Tokenize the test data
out_csv_path = path_to_tweets + 'token_test.csv'
df_test = bert_tokenize_test(df_test, out_csv_path, max_len=TOKEN_LEN)

In [20]:
# Compute and save the prediction
out_path = path_to_tweets + 'sub.txt'
prediction = get_prediction(model, df_test, BATCH_LEN, out_path, device)

100%|██████████| 500/500 [00:26<00:00, 18.71it/s]
